In [ ]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')
# from scipy import stats
# from keras.layers import Dense
# from keras.layers import Dropout
# from sklearn.utils import shuffle
# from keras.models import Sequential
# from sklearn.metrics import r2_score
# from matplotlib import pyplot as plt
# from sklearn.pipeline import Pipeline
# from keras.utils import to_categorical
# from keras.utils import multi_gpu_model
# from keras.models import model_from_json
# from sklearn.model_selection import KFold
# from keras.layers import BatchNormalization
# from sklearn.datasets import make_regression
# from sklearn.metrics import confusion_matrix 
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import mean_absolute_error
# from sklearn.metrics import mean_absolute_error
# from sklearn.preprocessing import StandardScaler
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import cross_val_score
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# from keras.wrappers.scikit_learn import KerasRegressor

In [ ]:
dataMain= pd.read_csv('sep19SexAndAgeAddedFINAL DATASET_ver2.txt') 
print(dataMain.shape)

dataMain=dataMain.query('FIELD_15 !=1 and FIELD_16 !=1 and FIELD_17 !=1 and FIELD_22 != 1 and FIELD_23 != 1 and FIELD_24 != 1 and FIELD_30 != 1')
print(dataMain.shape)

In [ ]:
newdata=dataMain.copy()
print(newdata.shape)
newdata.head()

In [ ]:
# for checking if there are two records per day
newdata_count_temp=newdata.groupby(by=['FIELD_1','FIELD_2']).size().reset_index(name='freq') 
# print(newdata_count_temp[newdata_count_temp.freq>1])

# group records by id and year to find out the number of tests per specific year
newdata_count_temp['Year']=(newdata_count_temp.FIELD_2 / 10000).astype(int)
newdata_count_peryear=newdata_count_temp.groupby(by=['FIELD_1','Year']).size().reset_index(name='freq')
# max(temp[temp.freq>1].freq)
newdata_count_peryear=newdata_count_peryear.sort_values(by='FIELD_1')
newdata_count_peryear.head()

# to get the number of years a patient visited
newdata_count_peryearG1= newdata_count_peryear.groupby(by=['FIELD_1']).size().reset_index(name='freq')
newdata_count_peryearG1.head()
print(newdata_count_peryearG1.freq.unique())
newdata_count_peryearG1=newdata_count_peryearG1[newdata_count_peryearG1.freq>1]
print(newdata_count_peryearG1.shape)
newdata_count_peryearG1.head()


In [ ]:
# newdata_count_temp[newdata_count_temp.freq>1].to_csv('DuplicateValues.csv',sep=',')
# newdata_count_temp.head()
# newdata_count_peryearG1.head()

In [ ]:
mylist=[]
for i in newdata_count_peryearG1.FIELD_1:
    vals=newdata_count_peryear[newdata_count_peryear.FIELD_1==i]
    mylist.append([i,np.sort(list(vals.Year))])

In [ ]:
mylistPD=pd.DataFrame(mylist,columns=['FIELD_1','Years'])
print(mylistPD.shape)
mylistPD.tail(10)

In [ ]:
newdata['Year']=(newdata.FIELD_2 / 10000).astype(int)

x=[]
target=[]

for _id in mylistPD.FIELD_1:
    ID=_id
    years=mylistPD[mylistPD.FIELD_1==ID].Years.values

    for y in range(len(years[0])-1):
        dif=years[0][y+1] - years[0][y]
        if(dif==1):
            
            tempdata=newdata.query('FIELD_1=='+str(ID)+' and Year=='+str(years[0][y])).reset_index()         
            x.append(tempdata.iloc[tempdata.shape[0]-1,:].values)

            tempdata=newdata.query('FIELD_1=='+str(ID)+' and Year=='+str(years[0][y+1])).reset_index()         
            target.append(tempdata.iloc[tempdata.shape[0]-1,:].values)



In [ ]:
newdata.head()

In [ ]:
# print(list(newdata.columns))

newcollist=newdata.columns
newcollist= np.insert(newcollist,0,'index')
newcollist

In [ ]:
newdata.shape
tempcols=newcollist
# print(list(newdata.columns))

In [ ]:
xlable=pd.DataFrame(x,columns=tempcols).drop(columns='Unnamed: 0')
xlable.tail(10)

In [ ]:
targelable=pd.DataFrame(target,columns=tempcols).drop(columns='Unnamed: 0')
targelable.tail(10)

In [ ]:
print(xlable.shape)
print(targelable.shape)

In [ ]:
targelable.to_csv('_targelable4_withNa_AllColumns.txt',sep=',')
xlable.to_csv('_xlable4_withNa_AllColumns.txt',sep=',')

In [ ]:
print(xlable.shape)
print(targelable.shape)

In [ ]:
testpd=pd.DataFrame()
testpd['test']=abs(xlable.FIELD_1 + xlable.Year - targelable.FIELD_1 - targelable.Year)

In [ ]:
testpd[testpd.test!=1]